# A one dimensional quantum well 

[Go back to index](./index.ipynb)

The widget displays interactively the eigenfunctions (plotted at the height of the corresponding eigenvalues) for a single quantum well, as obtained from the numerical solution. 

In [15]:
%matplotlib widget
from pylab import axes, setp, axis, xlabel, ylabel, title, plot, subplot, draw, connect, show
from numpy import linspace, sqrt, ones, arange, diag, argsort, zeros
from scipy.linalg import eig, eigh_tridiagonal
import matplotlib.pyplot as plt

colors = ['#9e0142','#d53e4f','#f46d43','#fdae61','#fee08b','#ffffbf','#e6f598','#abdda4','#66c2a5','#3288bd','#5e4fa2']    
colors = colors + colors + colors

def singlewell_potential(x, width, depth):
    x1 = zeros(len(x))
    for i in range(len(x)):
        if x[i] > - width/2.0 and x[i] < width/2.0:
            x1[i] = depth
    return x1
    

def diagonalisierung(hquer, L, N, pot=singlewell_potential, width = 0.1, depth = 0.0):
    """Calculated sorted eigenvalues and eigenfunctions. 

       Input:
         hquer: Planck constant
         L: set viewed interval [-L,L] 
         N: number of grid points i.e. size of the matrix 
         pot: potential function of the form pot
         x0: center of the quantum well
         width: the width of the quantum well
         depth: the depth of the quantum well
       Ausgabe:
         ew: sortierte Eigenwerte (Array der Laenge N)
         ef: entsprechend sortierte Eigenvektoren, ef[:,i] (Groesse N*N)
         x: Ortsgitter (Array der Laenge N)
         dx: Ortsgitterabstand
         V: Potential an den Stellen x (Array der Laenge N)
    """
    x = linspace(-L, L, N+2)[1:N+1]                 # grid points
    dx = x[1] - x[0]                                # grid spacing
    V = pot(x, width, depth)
    z = hquer**2 /2.0/dx**2                         # second diagonals

    ew, ef = eigh_tridiagonal(V+2.0*z, -z*ones(N-1))
    ew = ew.real                                    # real part of the eigenvalues
    ind = argsort(ew)                               # Indizes f. sort. Array
    ew = ew[ind]                                    # Sortieren der ew nach ind
    ef = ef[:, ind]                                 # Sortieren der Spalten von
                                                    #  ef nach ind
    ef = ef/sqrt(dx)                                # richtige Normierung 
    return ew, ef, x, dx, V


def plot_eigenfunktionen(ax, ew, ef, x, V, width=1, Emax=0.1, fak= 5.0):
    """Plot der niedrigsten Eigenfunktionen 'ef' im Potential 'V(x)'
       auf Hoehe der Eigenwerte 'ew' in den Plotbereich 'ax'.
       
       Der optionale Parameter 'width' (mit Defaultwert 1)
       gibt die Linienstaerke beim Plot der Eigenfunktionen
       an. 'width' kann auch ein Array von Linienstaerken sein.
       'Emax' (mit Default-Wert V_0/10) legt die Energieobergrenze
       fuer den Plot fest.
       'fak' ist ein Skalierungsfaktor fuer die graphische Darstellung
       der Eigenfunktionen.
    """
    
    fak = fak/100.0;
    
    ax[1].plot(x, V, c='r', linewidth=1.5)
    ax[0].plot(x, V, c='r', linewidth=1.5)
    
    ax[1].set_xlim([min(x), max(x)])
    ax[1].set_ylim([min(V)-0.05, Emax])
    
    ax[1].set_xlabel(r'$x/a$', fontsize = 10)
    ax[1].set_ylabel(r'$V(x)/V_0\ \rm{, Eigenfunctions\ with\ Eigenvalues}$', fontsize = 8)
    
    ax[0].set_xlim([min(x), max(x)])
    ax[0].set_ylim([min(V)-0.05, Emax])
    
    ax[0].set_xlabel(r'$x/a$', fontsize = 10)
    ax[0].set_ylabel(r'$V(x)/V_0\ \rm{, Eigenfunctions\ with\ Eigenvalues}$', fontsize = 8)
      
    indmax = sum(ew<=Emax)                       
    if not hasattr(width, "__iter__"):           
        width = width*ones(indmax)               
    for i in arange(indmax):                     
        ax[0].plot(x, fak*abs(ef[:, i])**2+ew[i], linewidth=width[i]+.1, color=colors[i%len(colors)])


In [15]:
from ipywidgets import FloatSlider, jslink, VBox, HBox

mu = 0.06                                            # Potentialparameter
L = 1.5                                              # x-Bereich ist [-L,L]
N = 200                                              # Zahl der Gitterpunkte
hquer = 0.06                                         # effektives hquer
sigma_x = 0.1                                        # Breite Gauss
zeiten = linspace(0.0, 10.0, 400)                    # Zeiten f. Zeitentw.


swidth = FloatSlider(value = 1.2, min = 0.1, max = 2.0, description = 'Width: ')
sdepth = FloatSlider(value = -0.2, min = -1.0, max = 0.0, step = 0.05, description = 'Depth: ')
sfak = FloatSlider(value = 5, min = 1.0, max = 10.0, step = 1.0, description = r'Zoom factor: ')

width = 1.2
depth = -0.2
fak = 5.0

ew, ef, x, dx, V = diagonalisierung(hquer, L, N, width = width, depth = depth)
    
fig, ax = plt.subplots(2, 1, figsize=(8,6))
fig.canvas.header_visible = False
#fig.canvas.layout.height = "800px"

fig.suptitle('Numerial Solution of One Dimension Schroedinger Equation', fontsize = 10)
plot_eigenfunktionen(ax, ew, ef, x, V)


def on_width_change(change):
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, width = swidth.value, depth = sdepth.value)
    plot_eigenfunktionen(ax, ew, ef, x, V, fak = sfak.value)

def on_depth_change(change):
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, width = swidth.value, depth = sdepth.value)
    plot_eigenfunktionen(ax, ew, ef, x, V, fak = sfak.value)
    
def on_xfak_change(change):
    for i in range(len(ax[0].lines)):
        ax[0].lines.pop(0)
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    plot_eigenfunktionen(ax, ew, ef, x, V, fak = sfak.value)
    
swidth.observe(on_width_change, names = 'value')
sdepth.observe(on_depth_change, names = 'value')
sfak.observe(on_xfak_change, names = 'value')

display(HBox([swidth, sdepth, sfak]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This work has been done with the support of the EPFL Open Science Fund [OSSCAR](http://www.osscar.org).

<img src="http://www.osscar.org/wp-content/uploads/2019/03/OSSCAR-logo.png" style="height:40px; width: 200px"/>